Problem 1: Please refer to the README to run the above implementation.

In [1]:
! pip install h5py==2.10.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached h5py-2.10.0.tar.gz (301 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for h5py
Failed to build h5py


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [74 lines of output]
      C:\Users\Work\AppData\Roaming\Python\Python311\site-packages\setuptools\__init__.py:81: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\h5py
      copying h5py\h5py_warnings.py -> build\lib.win-amd64-cpython-311\h5p

In [2]:
!python predict.py -i ./kaggle_simpson_testset -c .\train_20230420-231810_config.pickle

python: can't open file 'c:\\Users\\Work\\Documents\\GitHub\\test1\\predict.py': [Errno 2] No such file or directory


Problem 2: Code reading

- RPN: It generates candidate regions of interest (ROIs) based on the input image. RPN speeds up the calculation of area proposals, which has been a bottleneck in calculation speed.

- RoI pooling layer: A convolution layer is shared between RPN and R-CNN in order to improve the efficiency of calculation cost. Therefore, the RoI pooling layer outputs the proposed areas of various sizes in a fixed size.

- IOU: The IOU, which is the overlap between the ground truth and the anchor, is calculated to determine the proposed area.

    From the train.py

row 71th: model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)

load RPN model_rpn using Get_model from faster_rcnn.py

row 102th: R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_data_format(), use_regr=True, overlap_thresh=0.7, max_boxes=300)

RPN model training, estimation and sending to Roi pooling

The threshold is set to 0.7 and iou is calculated. The maximum number of proposal area boxes is set to 300.

Problem 3: Estimation by learned weights

In [3]:
!python ./yolo/convert.py ./yolo/yolov3.cfg ./yolo/yolov3.weights ./yolo/model_data/yolo_weights.h5

python: can't open file 'c:\\Users\\Work\\Documents\\GitHub\\test1\\yolo\\convert.py': [Errno 2] No such file or directory


Problem 4: Create a file for learning.
Learn new data (Simpsons dataset). Read the Training section of the README.md and create the necessary files to train the Simpsons dataset.

The format of the annotation file is different from the implementation of Problem 1, so it needs to be converted.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

annotation_df = pd.read_csv('annotation.txt',header=None)
print(annotation_df.head())

le = LabelEncoder()
annotation_df.iloc[:,5] = le.fit_transform(annotation_df.iloc[:,5])

                                                   0    1    2    3    4  \
0  simpsons_dataset/abraham_grampa_simpson/pic_00...   57   72   52   72   
1  simpsons_dataset/abraham_grampa_simpson/pic_00...   80   31  337  354   
2  simpsons_dataset/abraham_grampa_simpson/pic_00...  128   48  285  407   
3  simpsons_dataset/abraham_grampa_simpson/pic_00...   72  126  158  275   
4  simpsons_dataset/abraham_grampa_simpson/pic_00...  123   61  294  416   

                        5  
0  abraham_grampa_simpson  
1  abraham_grampa_simpson  
2  abraham_grampa_simpson  
3  abraham_grampa_simpson  
4  abraham_grampa_simpson  


In [2]:
#Converting annotation file format
rcnn_path = 'annotation_rcnn.txt'
yolo_path = 'annotation.txt'
n_sample, n_col = annotation_df.shape
with open(rcnn_path) as f:
    lines = f.readlines() 
    for i in range(n_sample):
        line = lines[i]
        split_line = line.split(',') 
        image_path = split_line[0]
        split_line[0] = './' + image_path
        split_line[-1] = str(annotation_df.iloc[i,5]) + '\n'  
        with open(yolo_path, mode='a') as out_f:
            join_line = ','.join(split_line)  
            join_line = join_line.replace('.jpg,','.jpg ')  
            out_f.write(join_line)

FileNotFoundError: [Errno 2] No such file or directory: 'annotation_rcnn.txt'

In [ ]:
annotation_df = pd.read_csv('annotation.txt',header=None)
print(annotation_df.head())

                                                   0    1    2    3  4
0  ./simpsons_dataset/abraham_grampa_simpson/pic_...   72   52   72  0
1  ./simpsons_dataset/abraham_grampa_simpson/pic_...   31  337  354  0
2  ./simpsons_dataset/abraham_grampa_simpson/pic_...   48  285  407  0
3  ./simpsons_dataset/abraham_grampa_simpson/pic_...  126  158  275  0
4  ./simpsons_dataset/abraham_grampa_simpson/pic_...   61  294  416  0


Problem 5: Confirmation that learning can be done.
Use the file you created in Problem 4 to learn. If it takes a long time to learn in the execution environment, you can just confirm that you can learn.

In [ ]:
!python ./yolo/train.py